<h1>Nltk</h1>

<a href="https://www.nltk.org/book/">Referência sobre Nltk</a>

O Nltk é uma biblioteca para processamento de texto natural em python. Com ela é possível realizar classificação de textos, treinar chunkers, entre outros

In [10]:
# -*- coding: utf-8 -*-

import nltk

Primeiramente, uma função bastante útil para o processamento de textos em português

In [66]:
from unicodedata import normalize

def normalize_text(text, to_lower=True):
    text = normalize('NFKD', text).encode('ASCII', 'ignore')

    final_text = text

    if to_lower:
        final_text = text.lower()
        
    return final_text.decode("utf-8")

In [67]:
print(normalize_text('OlÁ, tudo bem? como você vai?'))
print(normalize_text('Olá, tudo bem? como você vai?', to_lower=False))

ola, tudo bem? como voce vai?
Ola, tudo bem? como voce vai?


Ao realizar a normalização de textos, considere excluir casos como o verbo de ligação "é" e o símbolo de parágrafo &para;

<h2>Baixando corpora e pacotes úteis</h2
<br><br>
Vamos agora baixar exemplos de textos em português para trabalhar e alguns pacotes que serão úteis

In [ ]:
#Baixar: book
#Baixar: all-corpora
#Baixar: floresta
#Baixar: machado
#Baixar: punkt
#Baixar: stopwords

#Se houverem problemas com a interface. Abrir o terminal do Jupyter e inserir os comandos:
#python -m nltk downloader book
#python -m nltk downloader all-corpora
#python -m nltk.downloader floresta
#python -m nltk.downloader machado
#python -m nltk.downloader punkt
#python -m nltk.downloader stopwords

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


<h2>Carregando textos e conhecendo funções úteis do Nltk</h2>

<h2>Segmentação de texto</h2>

<h3>Tokenização</h3>
<br>
Tokenização é o processo de dividir uma frase em palavras particulares


In [46]:
nltk.word_tokenize('meu nome é felipe navarro')

['meu', 'nome', 'é', 'felipe', 'navarro']

<h3>Segmentação em sentenças (sent tokenization)</h3>
<br>
É o processo de quebrar um texto grande em frases individuais


In [52]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

texto = 'Meu nome é João da Silva e gosto de passear no feriado. Recentemente comprei um carro, que quebrou ontem'

sent_tokenizer.tokenize(texto)


['Meu nome é João da Silva e gosto de passear no feriado.',
 'Recentemente comprei um carro, que quebrou ontem']

In [53]:
#Para simplificar vamos criar funções

def tokens(my_str):
    return nltk.word_tokenize(my_str)

def sent_seg(my_str):
    return sent_tokenizer.tokenize(texto)

<h2>Classificação sintática</h2>

<h3>Visualizando corpus taggeado</h3>

In [21]:
from nltk.corpus import floresta
#Visualizando as palavras
print(floresta.words()[:10])

print('----------')
#Com tags
print(floresta.tagged_words()[:10])

['Um', 'revivalismo', 'refrescante', 'O', '7_e_Meio', 'é', 'um', 'ex-libris', 'de', 'a']
----------
[('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj'), ('O', '>N+art'), ('7_e_Meio', 'H+prop'), ('é', 'P+v-fin'), ('um', '>N+art'), ('ex-libris', 'H+n'), ('de', 'H+prp'), ('a', '>N+art')]


In [33]:
def simple_tag(tag):
    try:
        return tag.split('+')[1]
    except:
        return tag

simple_tag('>N+art')

'art'

In [27]:
#Observando por sentenças
tagged_sents = floresta.tagged_sents()

for sent in tagged_sents[:10]:
    print(sent)
    print('--------')

[('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj')]
--------
[('O', '>N+art'), ('7_e_Meio', 'H+prop'), ('é', 'P+v-fin'), ('um', '>N+art'), ('ex-libris', 'H+n'), ('de', 'H+prp'), ('a', '>N+art'), ('noite', 'H+n'), ('algarvia', 'N<+adj'), ('.', '.')]
--------
[('É', 'P+v-fin'), ('uma', 'H+num'), ('de', 'H+prp'), ('as', '>N+art'), ('mais', '>A+adv'), ('antigas', 'H+adj'), ('discotecas', 'H+n'), ('de', 'H+prp'), ('o', '>N+art'), ('Algarve', 'H+prop'), (',', ','), ('situada', 'P+v-pcp'), ('em', 'H+prp'), ('Albufeira', 'P<+prop'), (',', ','), ('que', 'SUBJ+pron-indp'), ('continua', 'AUX+v-fin'), ('a', 'PRT-AUX<+prp'), ('manter', 'MV+v-inf'), ('os', '>N+art'), ('traços', 'H+n'), ('decorativos', 'N<+adj'), ('e', 'CO+conj-c'), ('as', '>N+art'), ('clientelas', 'H+n'), ('de', 'H+prp'), ('sempre', 'P<+adv'), ('.', '.')]
--------
[('É', 'P+v-fin'), ('um_pouco', 'ADVL+adv'), ('a', '>N+art'), ('versão', 'H+n'), ('de', 'H+prp'), ('uma', '>N+art'), ('espécie', 'H+n'), ('de', 'H+prp'),

In [34]:
#Criando sentenças simplificadas
simplified_sents = [[(n, simple_tag(t)) for n, t in sent] for sent in tagged_sents]

In [35]:
simplified_sents[0]

[('Um', 'art'), ('revivalismo', 'n'), ('refrescante', 'adj')]

In [38]:
import random
random.shuffle(simplified_sents)

num_sents = len(simplified_sents)
len_train = int(num_sents*0.8)

train = simplified_sents[:len_train]
test = simplified_sents[len_train:]

<h3>Treinando POS taggers</h3>

In [43]:
from nltk.metrics import accuracy

tagger0 = nltk.DefaultTagger('n')

print('Acuracia default tagger', tagger0.evaluate(test))

tagger1 = nltk.UnigramTagger(train, backoff=tagger0)

print('Acuracia unigram tagger', tagger1.evaluate(test))

tagger2 = nltk.BigramTagger(train, backoff=tagger1)

print('Acuracia bigram tagger', tagger2.evaluate(test))

Acuracia default tagger 0.18786394306312967
Acuracia unigram tagger 0.8811119327109714
Acuracia bigram tagger 0.8945836029207875


<h3>Testando</h3>

In [48]:
tagger2.tag(tokens('eu sou felipe navarro'))

[('eu', 'pron-pers'), ('sou', 'v-fin'), ('felipe', 'n'), ('navarro', 'n')]

<h2>Chunkers</h2>

<h3>Contrução de uma base de dados para um Chunker em portugues</h3>

<a href="https://www.linguateca.pt/primeiroHAREM/harem_coleccaodourada_en.html">Fonte inicial</a>

In [69]:
from bs4 import BeautifulSoup as Soup

raw_xml = open('datasets/ColeccaoDouradaHAREM.txt', 'r', encoding='latin-1').read()
raw_xml = raw_xml.replace('|', ' ')
raw_xml = normalize_text(raw_xml)
print(raw_xml[:300])

<doc>
<docid>harem-871-07800</docid>
<genero>web</genero>
<origem>pt</origem>
<texto>
<organizacao tipo="instituicao" morf="f,s">abraco</organizacao> pagina principal
<organizacao tipo="instituicao" morf="f,s">associacao de apoio a pessoas com vih/sida</organizacao>
a <organizacao tipo="instituicao"


In [70]:
soup = Soup(raw_xml)
all_textos = soup.findAll('texto')
num_textos = len(all_textos)
print('Num textos', num_textos)
print('....')
texts_with_people = []

for ctext in all_textos:
    people = ctext.findAll('pessoa')
    if len(people) > 1:
        texts_with_people.append([ctext.text, [p.text for p in people]])

print('Num textos com pessoas', len(texts_with_people))

Num textos 129
....
Num textos com pessoas 78


In [71]:
texts_with_people[0]

['\nfernando ferreira\n[click for a page in english]\ncmaf- universidade de lisboa gabinete a2-31 avenida professor gama pinto, 2 telefone do gabinete: 217904893 p-1649-003 lisboa extensao interna: 4293 portugal email: ferferr@cii.fc.ul.pt departamento de matematica faculdade de ciencias universidade de lisboa cmaf\napresentacao\nbem vindos aminha pagina pessoal. sou professor associado do departamento de matematica da universidade de lisboa e membro do centro de matematica e aplicacoes fundamentais - cmaf. clique aqui para obter o meu cv.\ninteresses academicos\nlogica matematica, em especial teorias fracas da aritmetica e da analise. filosofia  e fundamentos de matematica . tenho um interesse amador (no sentido latino da palavra) por alguns problemas da filosofia antiga , particularmente no problema da falsidade em parmenides e platao. tambem escrevi alguns ensaios expositorios sobre temas da logica: clique aqui para os ver.\nensino\nno presente semestre dou aulas teorico-praticas de

In [72]:
import nltk
def create_tagged_people_names(name, tag_name):
    tokens = nltk.word_tokenize(name)
    first_token = tokens[0] +'|B-' +tag_name
    
    middle_tokens = tokens[1:]
    return ' '.join([first_token] + [t+'|I-'+tag_name for t in middle_tokens]) + ' '

create_tagged_people_names('fernando ferreira', 'nome')

def create_fully_tagged_text(full_text, names, tag_name):
    tagged_names = [create_tagged_people_names(n, tag_name) for n in names]
    modified_text = full_text
    
    for i, name in enumerate(names):
        modified_text = modified_text.replace(name, tagged_names[i])
        
    tokens = nltk.word_tokenize(modified_text)
    modified_tokens = []
    for t in tokens:
        if '|' in t:
            modified_tokens.append(t)
        else:
            modified_tokens.append(t + '|O')
    return ' '.join(modified_tokens)

def create_separated_tag_text(tagged_text):
    tokens = nltk.word_tokenize(tagged_text)
    return [x.split('|') for x in tokens if len(x.split('|')) > 1]

def process_entry(entry, tag_name):
    text = entry[0]
    names = entry[1]
    ftt = create_fully_tagged_text(text, names, tag_name)
    return create_separated_tag_text(ftt)

process_entry(texts_with_people[0], 'nome')

[['fernando', 'B-nome'],
 ['ferreira', 'I-nome'],
 ['', 'O'],
 ['click', 'O'],
 ['for', 'O'],
 ['a', 'O'],
 ['page', 'O'],
 ['in', 'O'],
 ['english', 'O'],
 ['', 'O'],
 ['cmaf-', 'O'],
 ['universidade', 'O'],
 ['de', 'O'],
 ['lisboa', 'O'],
 ['gabinete', 'O'],
 ['a2-31', 'O'],
 ['avenida', 'O'],
 ['professor', 'O'],
 ['gama', 'O'],
 ['pinto', 'O'],
 ['', 'O'],
 ['2', 'O'],
 ['telefone', 'O'],
 ['do', 'O'],
 ['gabinete', 'O'],
 ['', 'O'],
 ['217904893', 'O'],
 ['p-1649-003', 'O'],
 ['lisboa', 'O'],
 ['extensao', 'O'],
 ['interna', 'O'],
 ['', 'O'],
 ['4293', 'O'],
 ['portugal', 'O'],
 ['email', 'O'],
 ['', 'O'],
 ['ferferr', 'O'],
 ['', 'O'],
 ['cii.fc.ul.pt', 'O'],
 ['departamento', 'O'],
 ['de', 'O'],
 ['matematica', 'O'],
 ['faculdade', 'O'],
 ['de', 'O'],
 ['ciencias', 'O'],
 ['universidade', 'O'],
 ['de', 'O'],
 ['lisboa', 'O'],
 ['cmaf', 'O'],
 ['apresentacao', 'O'],
 ['bem', 'O'],
 ['vindos', 'O'],
 ['aminha', 'O'],
 ['pagina', 'O'],
 ['pessoal', 'O'],
 ['.', 'O'],
 ['sou', 'O'],

In [73]:
processed_entries = [process_entry(t, 'nome') for t in texts_with_people]

In [76]:
def create_features_for_words(sentence, index):
    current_word = sentence[index]
    prefix3 = current_word[:3]
    sufix3 = current_word[-3:]
    if index == 0:
        prev_word = ''
    else:
        prev_word = sentence[index - 1]
    feats = {'word': current_word,
            'prefix3': prefix3,
            'sufix3': sufix3,
            'prev_word': prev_word}
    return feats

def processed_entry_to_feats_and_targets(processed_entry):
    sentence = [w[0] for w in processed_entry]
    feats_and_targets = [[create_features_for_words(sentence, i), w[1]] for i, w in enumerate(processed_entry)]
    return feats_and_targets

new_processed = []
for pe in processed_entries:
    new_processed += processed_entry_to_feats_and_targets(pe)

In [75]:
Feats = [np[0] for np in new_processed]
Targets = [np[1] for np in new_processed]

In [77]:
set(Targets)

{'B-nome', 'I-nome', 'O'}

In [78]:
Feats = [np[0] for np in new_processed]
Targets = [np[1] for np in new_processed]

print('Len dataset', len(new_processed))

from sklearn.model_selection import train_test_split


F_train, F_test, T_train, T_test = train_test_split(Feats, Targets, 
                                                                    test_size=0.30, 
                                                                    stratify=Targets,
                                                                    shuffle=True)

Len dataset 74211


In [113]:
def convert_target(target):
    if target == 'O':
        return 0
    elif target == 'B-nome':
        return 1
    elif target == 'I-nome':
        return 2
    
def convert_target_index(target_index):
    if target_index == 0:
        return 'O'
    elif target_index == 1:
        return 'B-nome'
    elif target_index == 2:
        return 'I-nome'

T_train = [convert_target(t) for t in T_train]
T_test = [convert_target(t) for t in T_test]

In [88]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
X_train = vectorizer.fit_transform(F_train)
X_test = vectorizer.transform(F_test)

In [93]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced')

In [94]:
model.fit(X_train, T_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [95]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

prediction = model.predict(X_test)

cm = confusion_matrix(T_test, prediction)
print('Matriz de confusao', cm)

precision = precision_score(T_test, prediction, average=None)
recall = recall_score(T_test, prediction, average=None)

print('Precisao', precision)
print('Recall', recall)

Matriz de confusao [[20708   222    97]
 [  156   785    16]
 [  125    19   136]]
Precisao [0.98661203 0.76510721 0.54618474]
Recall [0.98482903 0.82027168 0.48571429]


In [96]:
#Garantindo o bom comportamento do modelo em termos de notacao IOB

def create_features_for_words_sl(sentence, index, previous_classification):
    current_word = sentence[index]
    prefix3 = current_word[:3]
    sufix3 = current_word[-3:]
    if index == 0:
        prev_word = ''
    else:
        prev_word = sentence[index - 1]
    feats = {'word': current_word,
            'prefix3': prefix3,
            'sufix3': sufix3,
            'prev_word': prev_word,
            'previous_classification': previous_classification}
    return feats

def processed_entry_to_feats_and_targets_sl(processed_entry):
    sentence = [w[0] for w in processed_entry]
    tags = [w[1] for w in processed_entry]
    
    previous_tag = 'O'
    feats_and_targets = []
    for i, w in enumerate(processed_entry):
        feats_and_targets.append(create_features_for_words_sl(sentence, i, previous_tag))
        previous_tag = tags[i]
    return feats_and_targets

new_processed_sl = []
for pe in processed_entries:
    new_processed_sl += processed_entry_to_feats_and_targets_sl(pe)

In [108]:
Feats_sl = [np[0] for np in new_processed]
Targets_sl = [np[1] for np in new_processed]

F_train_sl, F_test_sl, T_train_sl, T_test_sl = train_test_split(Feats_sl, Targets_sl, 
                                                                    test_size=0.30, 
                                                                    stratify=Targets_sl,
                                                                    shuffle=True)


T_train_sl = [convert_target(t) for t in T_train_sl]
T_test_sl = [convert_target(t) for t in T_test_sl]

vectorizer_sl = DictVectorizer()
X_train_sl = vectorizer_sl.fit_transform(F_train_sl)
X_test_sl = vectorizer_sl.transform(F_test_sl)

model_sl = LogisticRegression(class_weight='balanced')

model_sl.fit(X_train_sl, T_train_sl)

prediction_sl = model_sl.predict(X_test_sl)

cm_sl = confusion_matrix(T_test_sl, prediction_sl)
print('Matriz de confusao', cm_sl)

precision_sl = precision_score(T_test_sl, prediction_sl, average=None)
recall_sl = recall_score(T_test_sl, prediction_sl, average=None)

print('Precisao', precision_sl)
print('Recall', recall_sl)

Matriz de confusao [[20677   278    72]
 [  130   810    17]
 [  131    14   135]]
Precisao [0.98753463 0.73502722 0.60267857]
Recall [0.98335473 0.84639498 0.48214286]


In [116]:

def make_prediction_sl(text, classifier, vectorizer):
    tokens = nltk.word_tokenize(text)
    last_prediction = 'O'
    predictions = []
    for i, token in enumerate(tokens):
        feats = create_features_for_words_sl(tokens, i, last_prediction)
        vect_feats = vectorizer.transform([feats])
        prediction = convert_target_index(classifier.predict(vect_feats)[0])
        last_prediction = prediction
        predictions.append((token, prediction))
    return predictions



In [119]:
prediction = make_prediction_sl('o meu nome eh fernando carlos da silva, entendeu?', model_sl, vectorizer_sl)
print(prediction)

[('o', 'O'), ('meu', 'O'), ('nome', 'O'), ('eh', 'O'), ('fernando', 'O'), ('carlos', 'B-nome'), ('da', 'I-nome'), ('silva', 'I-nome'), (',', 'O'), ('entendeu', 'O'), ('?', 'O')]
